In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import warnings
import tqdm
warnings.filterwarnings('ignore')



In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [3]:
df_train = reduce_mem_usage(pd.read_csv('../input/train.csv',parse_dates=["first_active_month"]))
df_test = reduce_mem_usage(pd.read_csv('../input/test.csv',parse_dates=["first_active_month"]))
df_hist_trans = reduce_mem_usage(pd.read_csv('../input/historical_transactions.csv'))
df_new_merchant_trans = reduce_mem_usage(pd.read_csv('../input/new_merchant_transactions.csv'))


Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [4]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)

In [5]:
i=0
labels=[0, 1, 2, 3]
bins=[0, 3, 6, 9, 12]
day_bins=[0,10,20,30]
day_labels=[0,1,2]
install_bins=[-1, 3, 6, 999]
install_labels=[0, 1, 2]
month_bins=[-13,-10,-6,-3,0]
month_labels=[0,1,2,3]
q_list=[0, .25, .5, .75, 1.]
q_labels=[0,1,2,3]
dat_day=pd.to_datetime(pd.Series(['2019-02-26']))

def fe_for_transactions(df,i):
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df.sort_values(by='purchase_date', inplace=True)
    special=pd.to_datetime(pd.Series(['2018-02-14','2017-02-14','2018-01-01','2017-12-25']))
    if i==1:
        df['val_day']=(df['purchase_date']-special[0]).dt.days.apply(lambda x:1 if x<1 and x>-100 else 0)
    if i==0:
        df['val_day']=(df['purchase_date']-special[1]).dt.days.apply(lambda x:1 if x<1 and x>-100 else 0)
        df['val_day_2018']=(df['purchase_date']-special[0]).dt.days.apply(lambda x:1 if x<1 and x>-100 else 0)
    df['new_year']=(df['purchase_date']-special[2]).dt.days.apply(lambda x:1 if x<1 and x>-100 else 0)
    df['christmas_day']=(df['purchase_date']-special[3]).dt.days.apply(lambda x:1 if x<1 and x>-100 else 0)
    df['year'] = df['purchase_date'].dt.year
    #df['quarter']=df['purchase_date'].dt.quarter
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['day']=df['purchase_date'].dt.day
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0})
    df['category_3'] = df['category_3'].map({'A':0, 'B':1, 'C':2})
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((dat_day[0]- df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['season']=(pd.cut(df['month'], bins=bins, labels=labels)).cat.codes
    df['3div_day']=(pd.cut(df['day'], bins=day_bins, labels=day_labels)).cat.codes
    #df['is_install']=df['installments'].apply(lambda x: 0 if x==0 else 1)
    #df['install_period']=(pd.cut(df['installments'], bins=install_bins, labels=install_labels)).cat.codes
    df['month_lag_period']=(pd.cut(df['month_lag'], bins=month_bins, labels=month_labels)).cat.codes
    df['time_differ'] = df.groupby(['card_id'])['purchase_date'].agg(['diff']).astype('timedelta64[h]')
    #df['time_differ']=np.log1p((df['purchase_date'].shift(-1)-df['purchase_date'])/np.timedelta64(1,'h'))
    #df['state_count']=df.groupby('state_id')['card_id'].transform('count')
    #df['duration'] = df['purchase_amount']*df['month_diff']
    #df['amount_month_ratio'] = df['purchase_amount']/df['month_diff']
    #df['subsector_count']=df.groupby('subsector_id')['card_id'].transform('count')
    #df['amount_quantile']=(pd.qcut(df['purchase_amount'], q=q_list, labels=q_labels)).cat.codes
#     df['cat1_count']=df.groupby('category_1')['card_id'].transform('count')
#     df['cat2_count']=df.groupby('category_2')['card_id'].transform('count')
#     df['cat3_count']=df.groupby('category_3')['card_id'].transform('count')
    return df

In [6]:
df_hist_trans=fe_for_transactions(df_hist_trans,0)
df_new_merchant_trans=fe_for_transactions(df_new_merchant_trans, 1)

In [7]:
gc.collect()

21

In [8]:
aggs= {
    'df_hist_trans':
       {
           'purchase_amount':['max','min','mean','sum','var','median'],
           'installments':['max','min','sum','mean','std'],
           'purchase_date':['max', 'min'],
           'month_lag': ['max','min','mean','var'],
           'month_diff':['mean','std'],
           'authorized_flag':['sum','mean'],
           'weekend':['sum', 'mean'],
           'category_1':['sum', 'mean'],
           'category_3':['sum','mean'],
           'category_2':['sum','mean'],
           'card_id':['size'],
           'val_day':['mean'],
           #'new_year':['sum','mean'],
           'christmas_day':['sum','mean'],
           'val_day_2018':['sum','mean'],
           'season':['sum','mean','nunique'],
           '3div_day':['sum','mean'],
           'day':['mean','median'],
           'month_lag_period':['sum','mean'],
           #'install_period':['sum','mean'],
           #'is_install':['sum','mean'],
           'time_differ':['max','min','mean'],
           'month':['nunique','mean'],
           'hour':['nunique','mean'],
           'weekofyear':['nunique','mean'],
           'dayofweek':['nunique','mean'],
           #'subsector_id':['nunique'],
           'merchant_id':['nunique'],
           'merchant_category_id':['nunique'],
           #'city_id':['nunique'],
           'category_2_mean':['mean'],
           'category_3_mean':['mean'],
           #'state_count':['max','min','mean'],
           #'quarter':['mean','nunique'],
           #'amount_month_ratio':['mean','min','max','var'],
           #'duration':['mean','min','max','var']
           #'amount_quantile':['mean','sum']
       },
       'df_new_merchant_trans':
    {
           'purchase_amount':['max','min','mean','sum','var','median'],
           'installments':['max','min','sum','mean','std'],
           'purchase_date':['max', 'min'],
           'month_lag': ['max','min','mean','var'],
           'month_diff':['mean','std'],
           'authorized_flag':['sum','mean'],
           'weekend':['sum', 'mean'],
           'category_1':['sum', 'mean'],
           'category_3':['sum','mean'],
           'category_2':['sum','mean'],
           'card_id':['size'],
           'val_day':['mean'],
           'new_year':['sum','mean'],
           'christmas_day':['sum','mean'],
           'season':['sum','mean','nunique'],
           '3div_day':['sum','mean'],
           'day':['mean','median'],
           'month_lag_period':['sum','mean'],
           #'install_period':['sum','mean'],
           #'is_install':['sum','mean'],
           'time_differ':['max','min','mean'],
            'month':['nunique','mean'],
           'hour':['nunique','mean'],
           'weekofyear':['nunique','mean'],
           'dayofweek':['nunique','mean'],
           #'subsector_id':['nunique'],
           'merchant_id':['nunique'],
           'merchant_category_id':['nunique'],
           #'city_id':['nunique'],
           'category_2_mean':['mean'],
           'category_3_mean':['mean'],
           #'state_count':['max','min','mean'],
           #'quarter':['mean','nunique'],
           #'amount_month_ratio':['mean','min','max','var'],
           #'duration':['mean','min','max','var']
           #'amount_quantile':['mean','sum']
    }
    
}

In [9]:
def get_names(dic, name):
    cols=[name+'_'+k+'_'+i for k in dic.keys() for i in dic[k]]
    return cols

In [10]:
def before_agg(df):
    for col in ['category_2','category_3']:
        df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
    return df

def after_agg(df,name):
    df[name+'_purchase_date_diff'] = (df[name+'_purchase_date_max'] - df[name+'_purchase_date_min']).dt.days
    df[name+'_purchase_date_average'] = df[name+'_purchase_date_diff']/df[name+'_card_id_size']
    df[name+'_purchase_date_uptonow'] = (dat_day[0] - df[name+'_purchase_date_max']).dt.days
    df[name+'_time_differ_diff']=(df[name+'_time_differ_max']-df[name+'_time_differ_min'])
    #df[name+'_state_c_differ']=(df[name+'_state_count_max']-df[name+'_state_count_min'])
#     df[name+'_season_diff']=(df[name+'_season_max']-df[name+'_season_min'])
#     df[name+'_3div_day_diff']=(df[name+'_3div_day_max']-df[name+'_3div_day_min'])
    #df[name+'_install_period_diff']=(df[name+'_install_period_max']-df[name+'_install_period_min'])
    return df

In [11]:
df_hist_trans=before_agg(df_hist_trans)
df_hist_group=df_hist_trans.groupby('card_id').agg(aggs['df_hist_trans'])
df_hist_group.columns=get_names(aggs['df_hist_trans'],'hist')
df_hist_group.reset_index(drop=False,inplace=True)

df_hist_group=after_agg(df_hist_group,'hist')

df_train = df_train.merge(df_hist_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_group,on='card_id',how='left')


In [12]:
df_new_merchant_trans=before_agg(df_new_merchant_trans)
df_new_merc_group=df_new_merchant_trans.groupby('card_id').agg(aggs['df_new_merchant_trans'])
df_new_merc_group.columns=get_names(aggs['df_new_merchant_trans'],'new_hist')
df_new_merc_group.reset_index(drop=False,inplace=True)

df_new_merc_group=after_agg(df_new_merc_group,'new_hist')

df_train = df_train.merge(df_new_merc_group,on='card_id',how='left')
df_test = df_test.merge(df_new_merc_group,on='card_id',how='left')

In [13]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [14]:
def train_feats(df):
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (dat_day[0] - df['first_active_month']).dt.days
    df['hist_last_buy_dff']=(dat_day[0]-df['hist_purchase_date_max']).dt.days
    df['new_hist_last_buy_dff']=(dat_day[0]-df['new_hist_purchase_date_max']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']
#     df['day_usage_mean']=df['card_id_total']/df['elapsed_time']
#     df['total_size'] = df['new_hist_card_id_size']+df['hist_purchase_amount_sum']
    return df

In [15]:
df_train=train_feats(df_train)
df_test=train_feats(df_test)

for f in ['feature_1','feature_2','feature_3']:
    order_label = df_train.groupby([f])['outliers'].mean()
    df_train[f] = df_train[f].map(order_label)
    df_test[f] = df_test[f].map(order_label)
for f in ['feature_1','feature_2','feature_3']:
    df_train[f+'c']=df_train.groupby(f)['card_id'].transform('count')
    df_test[f+'c']=df_test.groupby(f)['card_id'].transform('count')

In [16]:
df_train_columns=[]
df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
# target = df_train['target']
# del df_train['target']

In [17]:

len(df_train_columns)

142

In [ ]:
df_train_columns=['hist_month_diff_mean',
 'new_hist_purchase_amount_max',
 'hist_authorized_flag_mean',
 'hist_category_1_sum',
 'new_hist_weekofyear_mean',
 'new_hist_day_mean',
 'hist_purchase_amount_median',
 'new_hist_month_lag_mean',
 'new_hist_purchase_amount_mean',
 'hist_category_1_mean',
 'hist_month_nunique',
 'hist_purchase_date_min',
 'new_hist_purchase_date_uptonow',
 'hist_time_differ_mean',
 'hist_purchase_date_max',
 'new_hist_hour_mean',
 'hist_hour_mean',
 'hist_first_buy',
 'hist_merchant_id_nunique',
 'hist_month_diff_std',
 'new_hist_purchase_date_diff',
 'new_hist_purchase_amount_var',
 'hist_purchase_amount_min',
 'new_hist_month_mean',
 'new_hist_time_differ_mean',
 'hist_purchase_date_diff',
 'hist_purchase_amount_mean',
 'new_hist_category_1_mean',
 'hist_val_day_mean',
 'hist_purchase_amount_max',
 'hist_weekofyear_nunique',
 'hist_month_lag_var',
 'new_hist_purchase_date_average',
 'hist_dayofweek_mean',
 'new_hist_time_differ_max',
 'hist_month_lag_mean',
 'hist_purchase_date_uptonow',
 'hist_installments_sum',
 'hist_3div_day_mean',
 'hist_purchase_date_average',
 'hist_category_2_mean_mean',
 #'hist_time_differ_min',
 'hist_purchase_amount_var',
 'new_hist_purchase_amount_min',
 'hist_time_differ_max',
 'hist_month_lag_period_mean',
 'new_hist_time_differ_min',
 'hist_weekend_mean',
 'hist_day_mean',
 'new_hist_purchase_amount_median',
 'hist_category_3_sum',
 'new_hist_first_buy',
 'hist_category_3_mean_mean',
 'elapsed_time',
 'hist_month_lag_period_sum',
 #'hist_state_count_mean',
 'hist_time_differ_diff',
 'new_hist_purchase_date_max',
 'hist_merchant_category_id_nunique',
 'hist_installments_std',
 'hist_purchase_amount_sum',
 'new_hist_purchase_amount_sum',
 'new_hist_category_1_sum',
 #'hist_is_install_sum',
 'hist_category_3_mean',
 'hist_val_day_2018_mean',
 'hist_season_sum',
 'hist_installments_mean',
 'new_hist_dayofweek_mean',
 'hist_3div_day_sum']
 #'hist_category_2_sum'
 #'hist_category_2_mean']

len(df_train_columns)

In [ ]:
df_train.to_csv("ReadytrainData.csv", index=False)
df_test.to_csv("ReadytestData.csv", index=False)